In [1]:
from ultralytics import YOLO
from clearml import Task

C:\Users\zlukich\AppData\Local\Programs\Python\Python39\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import optuna

In [3]:
path_to_data = 'C:/Users/zlukich/hyperparameter tuning/v8data/data.yaml'
model = YOLO('./runs/segment/train10/weights/best.pt')
#model.train(data = path_to_data,epochs = 10,batch = 64)

In [9]:
metrics = model.val()
#metrics = model.val()

Ultralytics YOLOv8.0.45  Python-3.9.7 torch-1.13.1+cpu CPU
YOLOv8n-seg summary (fused): 195 layers, 3258844 parameters, 0 gradients, 12.0 GFLOPs
val: Scanning C:\Users\zlukich\hyperparameter tuning\v8data\valid\labels... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00:00<00:00, 1442.62it/s]
val: New cache created: C:\Users\zlukich\hyperparameter tuning\v8data\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:12<00:00,  2.52s/it]
                   all         68        136     0.0108      0.667      0.606       0.57     0.0108      0.667      0.606      0.539
               Droplet         68         56     0.0065          1      0.995      0.928     0.0065          1      0.995       0.93
              Miniskus         68         12          0          0          0          0          0          0          0          0
               PipeJet  

In [10]:
metrics.box.map

0.5702293400364169

In [19]:
metrics = model.val()

Ultralytics YOLOv8.0.30  Python-3.10.2 torch-1.13.1+cpu CPU
YOLOv8n-seg summary (fused): 195 layers, 3258844 parameters, 0 gradients, 12.0 GFLOPs
val: Scanning C:\Users\zlukich\hyperparameter tuning\v8data\valid\labels.cache... 68 images, 0 backgrounds, 0 corrupt: 
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP
                   all         68        136    0.00583      0.621      0.391      0.257    0.00583      0.621      0.393      0.353
               Droplet         68         56    0.00827      0.982      0.968      0.649    0.00827      0.982      0.977      0.933
              Miniskus         68         12          0          0          0          0          0          0          0          0
               PipeJet         68         68    0.00921      0.882      0.203      0.122    0.00921      0.882      0.203      0.126
Speed: 1.8ms pre-process, 107.3ms inference, 0.0ms loss, 2.8ms post-process per im

In [ ]:
test_model = YOLO("yolov8n-seg.pt")  # load an official model

# Validate the model
metrics = test_model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps   # a list contains map50-95(B) of each category
metrics.seg.map    # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps   # a list contains map50-95(M) of each category

Ultralytics YOLOv8.0.30  Python-3.10.2 torch-1.13.1+cpu CPU
YOLOv8n-seg summary (fused): 195 layers, 3404320 parameters, 0 gradients, 12.6 GFLOPs
val: Scanning C:\Users\zlukich\PycharmProjects\yolo_test\datasets\coco\labels\val2017.cache... 4952 images, 48 backgrou
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP

In [15]:
import optuna

def objective(trial):
    
    batch = trial.suggest_int("batch",8,64,step = 8)
    
    imgsz = trial.suggest_int("imgsz",640,1280, step = 640)
    
    optimizer = trial.suggest_categorical("optimizer",['SGD', 'Adam', 'AdamW', 'RMSProp'])
    
    lr0 = trial.suggest_float("lr0",0.001,0.2)
    lrf = trial.suggest_float("lrf",0.001,0.2)
    
    model.train(data = path_to_data, batch = batch, imgsz = imgsz, optimizer = optimizer, lr0 = lr0, lrf = lrf,epochs = 1)
    
    return model.val().box.map

study = optuna.create_study()
study.optimize(objective,n_trials = 100)

best_params = study.best_params
print(best_params)

[I 2023-02-25 13:43:00,151] A new study created in memory with name: no-name-b878ca24-37b7-4c2f-8022-0a44b860e9da
Ultralytics YOLOv8.0.45  Python-3.9.7 torch-1.13.1+cpu CPU
yolo\engine\trainer: task=segment, mode=train, model=./runs/segment/train10/weights/best.pt, data=C:/Users/zlukich/hyperparameter tuning/v8data/data.yaml, epochs=1, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, min_memory=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, au

ClearML Task: created new task id=2234288e0526476b8b745a803458a063
ClearML results page: https://app.clear.ml/projects/090f3fa2cf244b94ae4c46a4b80c23b7/experiments/2234288e0526476b8b745a803458a063/output/log
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


optimizer: Adam(lr=0.03115735711337469) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias
train: Scanning C:\Users\zlukich\hyperparameter tuning\v8data\train\labels.cache... 236 images, 0 backgrounds, 0 corrupt: 100%|██████████| 236/236 [00:00<?, ?it/s]
val: Scanning C:\Users\zlukich\hyperparameter tuning\v8data\valid\labels.cache... 68 images, 0 backgrounds, 0 corrupt: 100%|██████████| 68/68 [00:00<?, ?it/s]
Plotting labels to runs\segment\train11\labels.jpg... 


2023-02-25 13:43:13,796 - clearml.Task - INFO - Storing jupyter notebook directly as code


Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\segment\train11
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size
  0%|          | 0/4 [00:51<?, ?it/s]
[W 2023-02-25 13:44:08,557] Trial 0 failed with parameters: {'batch': 64, 'imgsz': 640, 'optimizer': 'Adam', 'lr0': 0.03115735711337469, 'lrf': 0.02436308081953001} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\zlukich\AppData\Local\Programs\Python\Python39\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\zlukich\AppData\Local\Temp\ipykernel_14976\1406841286.py", line 14, in objective
    model.train(data = path_to_data, batch = batch, imgsz = imgsz, optimizer = optimizer, lr0 = lr0, lrf = lrf,epochs = 1)
  File "C:\Users\zlukich\AppData\Roaming\Python\Python39\site-packages\ultralytics\yolo\engine\mod

KeyboardInterrupt: 

In [12]:
model.train?